ADMM for $f(x) = \frac{1}{2}\|Ax-b\|^2_2+\langle x,v\rangle+\lambda \|x\|_1$<br />
can be reformulated as <br />
$\min \frac{1}{2}\|Ax-b\|^2_2+\langle x,v\rangle +\lambda \|z\|_1$ subject to $x-z=0$<br />
$\mathcal{L}_\delta(x,z,y)=\frac{1}{2}\|Ax-b\|_2^2+\langle x,v\rangle +\lambda \|z\|_1+y^T(x-z)+\frac{\delta}{2}\|x-z\|^2_2$
1. $x^{l+1} = \arg\min \mathcal{L}_\delta(x,z^l,y^l)$
2. $z^{l+1} = \arg\min \mathcal{L}_\delta(x^{l+1},z,y^l)$
3. $y^{l+1} = y^l+\delta(x^{l+1}-z^{l+1})$


In [69]:
import numpy as np
def generate_gauss_sparse(nrow, ncol, sparsity):
    A = np.random.randn(nrow,ncol)/np.sqrt(nrow)
    x = np.zeros(ncol)
    x[np.random.permutation(ncol)[:sparsity]]=1.0
    b = A.dot(x)
    return (A,x,b)
# A,b,v,lambda,delta
def square_l1_admm(A,b,lamb,delta=1e-4,v=None,errabs=1e-5,errrel=1e-5,maxit=5000,beta=None):
    nrow,ncol=A.shape
    if beta is None:
        beta = delta
    if v is None:
        v = np.zeros(ncol)
    def target(x):
        r = A.dot(x)-b
        return r.dot(r)+x.dot(v)+lamb*sum(abs(x))
    def shrink(x,r):
        return np.sign(x)*np.array(map(lambda x: max(abs(x)-r,0),x))
    
    ATb = A.T.dot(b)
    #print "A.shape="+str(A.shape)
    AATinv = np.linalg.inv(A.T.dot(A)+np.diag(delta*np.ones(ncol)))
    x = ATb.copy()
    z = x.copy()
    y = np.zeros(ncol)
    for i in range(maxit):
        x = AATinv.dot(ATb-v+delta*z-y)
        zold = z.copy()
        z = shrink(x+y/delta,lamb/delta)
        y = y+beta*(x-z)
        if i%10==0:
            r = x - z
            s = delta*(z-zold)
            if np.linalg.norm(r)<np.sqrt(ncol)*errabs+errrel*max(np.linalg.norm(x),np.linalg.norm(z)) and \
            np.linalg.norm(s)<np.sqrt(ncol)*errabs+errrel*np.linalg.norm(y):
                break
    return x

def dc_admm(A,b,lamb,rel=1e-2):
    x = A.T.dot(b)
    maxoit=10
    for i in range(maxoit):
        xold = x.copy()
        if np.linalg.norm(x)>1e-7:            
            x = square_l1_admm(A,b,lamb,delta=10*lamb,v=-lamb*x/np.linalg.norm(x),errabs=1e-7,errrel=1e-5,maxit=5000)
        else:
            x = square_l1_admm(A,b,lamb,delta=10*lamb,errabs=1e-7,errrel=1e-5,maxit=5000)
        if np.linalg.norm(x-xold)/max(np.linalg.norm(xold),1)<rel:
                break
    return x

def dc_fbs(A,b,lamb = 1e-2, rel = 1e-8, step = 1e-2, maxit = 100000):
    x = A.T.dot(b)
    def shrink(x,r):
        return np.sign(x)*np.array(map(lambda x: max(abs(x)-r,0),x))
    for i in range(maxit):
        xold = x.copy()
        # forward step
        if np.linalg.norm(x,1)>1e-6:
            x = x-(2*A.T.dot(A.dot(x)-b)-lamb*x/np.linalg.norm(x))*step
        else:
            x = x-2*A.T.dot(A.dot(x)-b)*step
        # backward step
        x = np.sign(x)*np.array(map(lambda x: max(abs(x)-step*lamb,0),x))
        if np.linalg.norm(x-xold,1)/max(np.linalg.norm(xold),1)<rel:
            break
    return x
    
def test():    
    nrow = 64
    ncol = 256
    sparsity = 15
    repeat = 10
    sparsitytable = range(10,41)
    result = np.zeros([2,len(sparsitytable)])
    for i in range(repeat):
        print "begin repeat "+str(repeat)
        for idxsparsity in range(len(sparsitytable)):
            sparsity = sparsitytable[idxsparsity]
            A,x,b = generate_gauss_sparse(nrow,ncol,sparsity)
            x_it = square_l1_admm(A,b,lamb=1e-6,errabs=1e-7,errrel=1e-5)
            if np.linalg.norm(x_it-x)/np.linalg.norm(x)<1e-3:
                result[0,idxsparsity]+=1
            x_it = dc_admm(A,b,1e-8)
            if np.linalg.norm(x_it-x)/np.linalg.norm(x)<1e-3:
                result[1,idxsparsity]+=1
    

In [72]:
A,x,b = generate_gauss_sparse(nrow,ncol,10)
x_it = square_l1_admm(A,b,lamb=1e-6,errabs=1e-7,errrel=1e-5)
if np.linalg.norm(x_it-x)/np.linalg.norm(x)<1e-3:
    print "admm ok"
else:
    print "admm failed"

x_it = dc_admm(A,b,1e-8)
if np.linalg.norm(x_it-x)/np.linalg.norm(x)<1e-3:
    print "dc ok"
else:
    print "dc failed"

x_it = dc_fbs(A,b,1e-2,rel=1e-5)
if np.linalg.norm(x_it-x)/np.linalg.norm(x)<1e-2:
    print "fbs ok"
else:
    print "fbs failed"


admm ok
dc ok
fbs ok


$f(x)=\|Ax-b\|^2_2+\lambda(\|x\|_1-\|x\|_2)$
该问题应该分为三个阶段来完成,后一阶段很大程度上依赖于上一阶段的正确性
1. Forward-backward (python)
1. randomized coodinate block forward-backward (python)
1. async coodinate block forward-backward (c++)